Working 03/01/2025

# Langchain LangGraph Jira Toolkit

This notebook goes over how to use the `Jira` toolkit.

The `Jira` toolkit allows agents to interact with a given Jira instance, performing actions such as searching for issues and creating issues, the tool wraps the atlassian-python-api library, for more see: https://atlassian-python-api.readthedocs.io/jira.html

## Installation and setup

To use this tool, you must first set as environment variables:
    JIRA_API_TOKEN
    JIRA_USERNAME
    JIRA_INSTANCE_URL
    JIRA_CLOUD

In [ ]:
%pip install --upgrade --quiet  atlassian-python-api

In [ ]:
%pip install -qU langchain-community langchain_openai langgraph

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.agent_toolkits.jira.toolkit import JiraToolkit
from langchain_community.utilities.jira import JiraAPIWrapper
from langchain_openai import ChatOpenAI
import os
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Dict, List, Tuple, Any, TypedDict, Optional
from langgraph.graph import StateGraph, END

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
JIRA_API_TOKEN = os.getenv("JIRA_API_TOKEN")
JIRA_USERNAME = os.getenv("JIRA_USERNAME")
JIRA_INSTANCE_URL = os.getenv("JIRA_INSTANCE_URL")
JIRA_CLOUD = os.getenv("JIRA_CLOUD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [8]:
os.environ["JIRA_API_TOKEN"] = JIRA_API_TOKEN
os.environ["JIRA_USERNAME"] = JIRA_USERNAME
os.environ["JIRA_INSTANCE_URL"] = JIRA_INSTANCE_URL
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["JIRA_CLOUD"] = JIRA_CLOUD

In [9]:
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0,max_tokens=16384)
jira = JiraAPIWrapper()
toolkit = JiraToolkit.from_jira_api_wrapper(jira)
jira_tools = toolkit.get_tools()

## Tool usage

Let's see what individual tools are in the Jira toolkit:

In [10]:
# Define the state schema for LangGraph
class AgentState(TypedDict):
    messages: List[Any]  # Store conversation messages
    tools: List[Any]  # Available tools

In [11]:
# Create the agent prompt (similar to ZERO_SHOT_REACT_DESCRIPTION's prompt)
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that can use Jira tools to help users manage their Jira instance.

You have access to the following tools:

{tools}

Use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original question"""),
    MessagesPlaceholder(variable_name="messages"),
])

In [12]:
# Define the agent node function for ChatOpenAI
def agent_node(state: AgentState) -> dict:
    # Get available tools and prepare tool info for the prompt
    tools = state["tools"]
    tool_names = [tool.name for tool in tools]
    tool_descs = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

    # Format the messages for the agent
    messages = state["messages"]

    # Execute the agent
    formatted_prompt = prompt.format_messages(
        messages=messages,
        tools=tool_descs,
        tool_names=", ".join(tool_names)
    )

    response = llm.invoke(formatted_prompt)

    # Return updated state with the agent's response
    return {"messages": messages + [response]}

In [13]:
# Define the function to route based on agent's output
def route_actions(state: AgentState) -> str:
    # Get the last message (which should be from the assistant)
    last_message = state["messages"][-1]
    if not isinstance(last_message, AIMessage):
        return "agent"

    # Check if the message contains "Final Answer:"
    if "Final Answer:" in last_message.content:
        return END

    # Check if the message has "Action:" and "Action Input:"
    if "Action:" in last_message.content and "Action Input:" in last_message.content:
        return "action"

    # Default back to the agent if no clear directive
    return "agent"

In [14]:
# Define function to execute tools
def execute_tools(state: AgentState) -> Dict:
    # Get the last assistant message
    last_message = state["messages"][-1]

    # Extract action and action input
    content = last_message.content
    action_start = content.find("Action:") + len("Action:")
    action_end = content.find("\n", action_start)
    action_input_start = content.find("Action Input:") + len("Action Input:")
    action_input_end = content.find("\n", action_input_start)

    if action_input_end == -1:  # If this is the last line
        action_input_end = len(content)

    action = content[action_start:action_end].strip()
    action_input = content[action_input_start:action_input_end].strip()

    # Find the appropriate tool
    for tool in state["tools"]:
        if tool.name == action:
            # Execute the tool
            try:
                observation = tool.invoke(action_input)
            except Exception as e:
                observation = f"Error: {str(e)}"
            break
    else:
        observation = f"Tool '{action}' not found. Available tools: {[t.name for t in state['tools']]}"

    # Create a new message with the observation
    observation_message = HumanMessage(content=f"Observation: {observation}")

    # Return the updated state
    return {"messages": state["messages"] + [observation_message]}

In [15]:
# Build the graph
def build_jira_langgraph_agent():
    # Set up the agent state graph
    workflow = StateGraph(AgentState)

    # Add the nodes
    workflow.add_node("agent", agent_node)
    workflow.add_node("action", execute_tools)

    # Add the edges
    workflow.add_conditional_edges("agent", route_actions, {
        "action": "action",
        END: END,
    })
    workflow.add_edge("action", "agent")

    # Set the entry point
    workflow.set_entry_point("agent")

    # Compile the graph
    graph = workflow.compile()

    return graph

In [16]:
# Initialize the LangGraph agent
jira_graph = build_jira_langgraph_agent()

In [17]:
# Function to run the agent with a query
def run_jira_agent(query: str):
    # Initialize the state
    state = {
        "messages": [HumanMessage(content=query)],
        "tools": jira_tools
    }
    # Run the graph
    result = jira_graph.invoke(state)
    # Extract the final answer from messages
    messages = result["messages"]
    for message in reversed(messages):
        if isinstance(message, AIMessage) and "Final Answer:" in message.content:
            final_answer_start = message.content.rfind("Final Answer:") + len("Final Answer:")
            return message.content[final_answer_start:].strip()
    # If no final answer found, return the last message
    return messages[-1].content if messages else "No response generated"

In [18]:
os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [19]:
# Example: Run the agent with your query
result = run_jira_agent("give me a list of projects")
print(result)

Here are the projects:

1. **EchoSilo-kpmo** (Key: EKPMO, Type: Software)
2. **EchoSilo-pmo** (Key: EPMO1, Type: Business)
3. **ECHOSILO-SCRUM** (Key: ESCRUM, Type: Software)
4. **Jira Training Company Team Scrum** (Key: JTCTS, Type: Software)
5. **Jira Training Team** (Key: JTT, Type: Software, Style: Next-Gen)
6. **Jira Training Top-Level Team** (Key: JTTLT, Type: Software)
7. **Jira Training Team Scrum** (Key: JTTS, Type: Software, Style: Next-Gen)


In [ ]:
# Example: Run the agent with your query
result = run_jira_agent("how many user stories are in ESCRUM project?")
print(result)

In [ ]:
# Example: Run the agent with your query
result = run_jira_agent("how many user story points are in ESCRUM project?")
print(result)